### The Assembler

In [ ]:
class SimpleAssembler:
    # mnemonic → (opcode_byte, length_in_bytes, mode)
    # mode: "none"=no operand, "imm"=immediate literal, "addr"=label/address
    OPCODES = {
        "ADD B": (0x02, 1, "none"),
        "ADD C": (0x03, 1, "none"),
        "ADC B": (0x04, 1, "none"),
        "SUB B": (0x05, 1, "none"),
        "SUB C": (0x06, 1, "none"),
        "SUC B": (0x07, 1, "none"),
        "CMP":   (0x08, 1, "none"),
        "INR A": (0x09, 1, "none"),
        "INR B": (0x0A, 1, "none"),
        "INR C": (0x0B, 1, "none"),
        "DCR A": (0x0C, 1, "none"),
        "DCR B": (0x0D, 1, "none"),
        "DCR C": (0x0E, 1, "none"),

        "ANA B": (0x0F, 1, "none"),
        "ANA C": (0x10, 1, "none"),
        "TST":   (0x11, 1, "none"),
        "ORA B": (0x12, 1, "none"),
        "ORA C": (0x13, 1, "none"),
        "XRA B": (0x14, 1, "none"),
        "XRA C": (0x15, 1, "none"),
        "CMA":   (0x16, 1, "none"),
        "ANI":   (0x17, 2, "imm"),
        "ORI":   (0x18, 2, "imm"),
        "XRI":   (0x19, 2, "imm"),

        "MOV A,B": (0x1A, 1, "none"),
        "MOV A,C": (0x1B, 1, "none"),
        "MOV B,A": (0x1C, 1, "none"),
        "MOV B,C": (0x1D, 1, "none"),
        "MOV C,A": (0x1E, 1, "none"),
        "MOV C,B": (0x1F, 1, "none"),

        "MVI A": (0x20, 2, "imm"),
        "MVI B": (0x21, 2, "imm"),
        "MVI C": (0x22, 2, "imm"),

        "LDA":   (0x23, 2, "addr"),
        "LDB":   (0x24, 2, "addr"),
        "LDC":   (0x25, 2, "addr"),
        "STA":   (0x26, 2, "addr"),
        "LDA C": (0x29, 1, "none"),
        "STA C": (0x2A, 1, "none"),

        "JMP":   (0x2B, 2, "addr"),
        "JS":    (0x2C, 2, "addr"),
        "JZ":    (0x2D, 2, "addr"),
        "JC":    (0x2E, 2, "addr"),
        "JV":    (0x2F, 2, "addr"),

        "CALL":  (0x30, 2, "addr"),
        "RET":   (0x31, 1, "none"),
        "NOP":   (0x3E, 1, "none"),
        "HLT":   (0x3F, 1, "none"),
    }

    # Pretty-print ints as 0xHH when you print the list, while staying real ints.
    class _HexInt(int):
        def __repr__(self):
            return f"0x{self:02X}"

    def _split_mnemonic(self, line):
        """
        Return (mnemonic_key, operand_text) by matching longest key.
        Case-insensitive; commas normalized.
        """
        code = line.split(';', 1)[0].strip().upper()
        code = code.replace(', ', ',').replace(' ,', ',')
        for key in sorted(self.OPCODES.keys(), key=len, reverse=True):
            if code.startswith(key):
                rest = code[len(key):].strip()
                return key, rest
        raise ValueError(f"Unknown instruction in line: {line!r}")

    def _normalize_source(self, source):
        """Return list of dicts: [{'raw','code','comment'}] preserving comments."""
        if isinstance(source, str):
            lines = source.splitlines()
        else:
            lines = source
        out = []
        for raw in lines:
            code, sep, comment = raw.partition(';')
            code = code.rstrip()
            comment = comment.strip() if sep else ""
            # keep empty code lines only if they carry a standalone comment? (skip here)
            if code.strip() or comment:  # we only keep non-empty code (comments kept attached)
                out.append({"raw": raw, "code": code.strip(), "comment": comment})
        return out

    def assemble(self, source):
        """Assemble and return the raw integer bytes (for the emulator)."""
        machine, _listing = self.assemble_with_listing(source)
        return machine

    def assemble_with_listing(self, source):
        """
        Assemble and return (machine_bytes, listing_text).
        - machine_bytes: List[int] (pretty-print as 0xHH if printed)
        - listing_text:  A human-readable listing with addresses and comments
        """
        lines = self._normalize_source(source)

        # First pass: compute addresses for labels; also classify lines
        labels = {}
        addr = 0
        classified = []  # list of dicts with type: 'label'|'instr'|'data'
        for entry in lines:
            code = entry["code"]
            if not code:
                continue
            if code.endswith(':'):
                label_name = code[:-1].strip().upper()
                labels[label_name] = addr
                classified.append({**entry, "type": "label", "name": label_name})
            else:
                # instruction or data?
                try:
                    inst, operand = self._split_mnemonic(code)
                    _, length, _ = self.OPCODES[inst]
                    classified.append({**entry, "type": "instr", "inst": inst, "operand": operand})
                    addr += length
                except ValueError:
                    # must be a data byte literal
                    try:
                        int(code, 0)
                    except ValueError:
                        raise ValueError(f"Line not instruction or data: {entry['raw']!r}")
                    classified.append({**entry, "type": "data"})
                    addr += 1

        # Second pass: emit bytes + build listing
        machine = []
        listing_lines = []
        addr = 0
        pending_label_comment = None  # attach label's comment to the very next data byte if that data line has none

        for entry in classified:
            t = entry["type"]
            if t == "label":
                pending_label_comment = entry["comment"] or None
                continue

            if t == "data":
                val = int(entry["code"], 0)
                if not (0 <= val <= 0xFF):
                    raise ValueError(f"Data byte out of range: {entry['raw']!r}")
                machine.append(self._HexInt(val))
                comment = entry["comment"] or pending_label_comment or ""
                listing_lines.append(f"{addr:02X}: 0x{val:02X}" + (f" ; {comment}" if comment else ""))
                addr += 1
                pending_label_comment = None
                continue

            # instruction
            inst = entry["inst"]
            operand_text = entry["operand"]
            opcode, length, mode = self.OPCODES[inst]

            # Emit opcode byte (comment attached to the opcode line)
            machine.append(self._HexInt(opcode))
            comment = entry["comment"] or ""
            listing_lines.append(f"{addr:02X}: 0x{opcode:02X}" + (f" ; {comment}" if comment else ""))
            addr += 1

            if length == 2:
                if not operand_text:
                    raise ValueError(f"Missing operand for '{inst}'")

                if mode == "imm":
                    try:
                        val = int(operand_text, 0)
                    except ValueError:
                        raise ValueError(f"Immediate expected for '{inst}', got: {operand_text!r}")
                else:  # addr mode
                    if operand_text in labels:
                        val = labels[operand_text]
                    else:
                        try:
                            val = int(operand_text, 0)
                        except ValueError:
                            raise ValueError(f"Label/address expected for '{inst}', got: {operand_text!r}")

                if not (0 <= val <= 0xFF):
                    raise ValueError(f"Operand out of range for '{inst}': {operand_text!r}")

                machine.append(self._HexInt(val))
                # Typically no comment on the operand byte line (to match your example)
                listing_lines.append(f"{addr:02X}: 0x{val:02X}")
                addr += 1

            # labels' pending comment does NOT apply to instructions; only to subsequent data
            pending_label_comment = None

        return list(machine), "\n".join(listing_lines)

### The simple CPU emulator

In [ ]:
class SimpleCPUEmulator:

    # Fixed class-level dispatch table. Shared by all instances of the SimpleCPUEmulator
    dispatch_table = {}
    
    @classmethod
    def opcode(cls, code):
        def decorator(func):
            cls.dispatch_table[code] = func
            return func
        return decorator


    def __init__(self):
        self.memory = [0] * 256 # Assuming 256 memory locations
        self.ip = 0             # The instruction pointer
        
        # The general purpose registers
        self.register_A = 0
        self.register_B = 0
        self.register_C = 0

        # The flags
        self.flag_Z = False
        self.flag_S = False
        self.flag_V = False
        self.flag_C = False
        
        # Controling the machine
        self.halted = False
        self.step_by_step = False

    def read_into_memory(self, program, start_address=0x00):
        """Load a list of byte-values into memory at the given start address."""
        end = start_address + len(program)
        if end > len(self.memory):
            raise ValueError(f"Program (size {len(program)}) exceeds memory bounds at {start_address:02X}.")
        # Copy program bytes in; leave the rest untouched
        for offset, byte in enumerate(program):
            self.memory[start_address + offset] = byte

    def memory_dump(self, start=0x00, end=None):
        """Print a formatted hex dump of memory from start to end (exclusive)."""
        if end is None or end > len(self.memory):
            end = len(self.memory)
        lines = []
        for addr in range(start, end, 16):
            chunk = self.memory[addr:addr+16]
            hex_bytes = ' '.join(f"{b:02X}" for b in chunk)
            lines.append(f"{addr:02X}: {hex_bytes}")
        print("\n".join(lines))
    
    # Updating the flags after executing an ALU operation
    def update_flags(self, total, result, op1, op2):
        
        self.flag_Z = 1 if result == 0 else 0
        self.flag_S = 1 if (result & 0x80) != 0 else 0
        self.flag_C = 1 if total > 0xFF else 0

        msb_op1 = (op1 & 0x80) >> 7
        msb_op2 = (op2 & 0x80) >> 7
        msb_result = (result & 0x80) >> 7
        self.flag_V = 1 if (msb_op1 == msb_op2 and msb_result != msb_op1) else 0

    # Displaying the current state of the machine
    def display_current_state(self):
        # Format registers as two-digit hex values
        registers = f"A: {self.register_A:02X}  B: {self.register_B:02X}  C: {self.register_C:02X}"
    
        # Format flags as 0 or 1 (assuming flags are booleans; otherwise, adjust accordingly)
        flags = f"Z: {int(self.flag_Z)}  S: {int(self.flag_S)}  V: {int(self.flag_V)}  C: {int(self.flag_C)}"
    
        # Format the memory content at the current program counter as a two-digit hex value
        #memory_content = f"{self.memory[self.ip]:02X}"
    
        # Print the formatted output
        print(f"Instruction pointer: {self.ip:02X}")
        print(f"Registers: {registers}")
        print(f"Flags:     {flags}")
        # later don't use len, memory size will be fixed
        if(self.ip) < len(self.memory):
            print(f"Memory Content at IP ({self.ip:02X}): {self.memory[self.ip]:02X}")
        else:
            print(f"Reached end of Memory!")

    # Running the program
    def run(self):
        ans = input("Run step-by-step? (y/N) ").strip().lower()
        if ans == 'y':
            self.run_step_by_step()
        else:
            self.run_full()
    
    def run_full(self):
        while not self.halted and 0 <= self.ip < len(self.memory):
            opcode = self.memory[self.ip]
            self.ip += 1

            operation = self.dispatch_table.get(opcode)
            if operation is None:
                raise Exception(f"Invalid opcode @ {self.ip-1:02X}: {opcode:02X}")
            operation(self)

    def run_step_by_step(self):
        # step loop
        while not self.halted and 0 <= self.ip < len(self.memory):
            opcode = self.memory[self.ip]
            self.ip += 1
            operation = self.dispatch_table.get(opcode)
            if operation is None:
                raise Exception(f"Invalid opcode @ {self.ip-1:02X}: {opcode:02X}")
            operation(self)

            # show state
            self.display_current_state()

            # interactive menu
            while True:
                print("\nOptions: [N]ext  [R]un to end  [D]ump memory  [I]nspect addr  [W]rite addr")
                choice = input("Choice (default N): ").strip().lower() or 'n'
                if choice == 'n':
                    # do one more step
                    break
                elif choice == 'r':
                    # finish the program in full
                    self.run_full()
                    return
                elif choice == 'd':
                    self.memory_dump()
                elif choice == 'i':
                    addr_s = input("  Address (hex or dec)? ")
                    try:
                        addr = int(addr_s, 0)
                        if 0 <= addr < len(self.memory):
                            print(f"  [{addr:02X}] = {self.memory[addr]:02X}")
                        else:
                            print("  >> out of range")
                    except ValueError:
                        print("  >> bad number")
                elif choice == 'w':
                    addr_s = input("  Address (hex or dec)? ")
                    val_s  = input("  New value (hex or dec)? ")
                    try:
                        addr = int(addr_s, 0)
                        val  = int(val_s, 0) & 0xFF
                        if 0 <= addr < len(self.memory):
                            self.memory[addr] = val
                            print(f"  Wrote {val:02X} to [{addr:02X}]")
                        else:
                            print("  >> address out of range")
                    except ValueError:
                        print("  >> bad input")
                else:
                    print("  >> unknown option")
        print("Execution finished.")

#import opcodes

### The opcodes

In [ ]:
# opcodes.py

# Import the Emulator class
#from simple_cpu_emulator import SimpleCPUEmulator

def opcode(code):
    def decorator(func):
        SimpleCPUEmulator.dispatch_table[code] = func
        return func
    return decorator

################
### Arithmetic
################

@opcode(0x02)
def opcode_ADD_B(self):
    print("Executing opcode: ADD B")
    op1 = self.register_A
    op2 = self.register_B
    total = op1 + op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags with the original operands for addition.
    self.update_flags(total, result, op1, op2)

@opcode(0x03)
def opcode_ADD_C(self):
    print("Executing opcode: ADD C")
    op1 = self.register_A
    op2 = self.register_C
    total = op1 + op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags with the original operands for addition.
    self.update_flags(total, result, op1, op2)

# Add with carry. Necessary when doing multi-byte additions
@opcode(0x04)
def opcode_ADC_B(self):
    print("Executing opcode: ADC B")
    op1 = self.register_A
    op2 = self.register_B
    total = op1 + op2 + 1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags with the original operands for addition.
    self.update_flags(total, result, op1, op2)

@opcode(0x05)
def opcode_SUB_B(self):
    print("Executing opcode: SUB B")
    op1 = self.register_A
    # Compute two's complement of register_B.
    neg_B = (~self.register_B) & 0xFF
    total = op1 + neg_B + 1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and neg_B (i.e., the effective second operand).
    self.update_flags(total, result, op1, neg_B)

@opcode(0x06)
def opcode_SUB_C(self):
    print("Executing opcode: SUB C")
    op1 = self.register_A
    # Compute two's complement of register_C.
    neg_C = (~self.register_C) & 0xFF
    total = op1 + neg_C + 1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and neg_B (i.e., the effective second operand).
    self.update_flags(total, result, op1, neg_C)

# Subtract with borrow. Necessary when doing multi-byte subtractions
@opcode(0x07)
def opcode_SUC_B(self):
    print("Executing opcode: SUC B")
    op1 = self.register_A
    # Compute two's complement of register_B.
    neg_B = (~self.register_B) & 0xFF
    total = op1 + neg_B # borrow is taken into account
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and neg_B (i.e., the effective second operand).
    self.update_flags(total, result, op1, neg_B)

@opcode(0x08)
def opcode_CMP(self):
    print("Executing opcode: CMP")
    op1 = self.register_A
    # Compute two's complement of register_B.
    neg_B = (~self.register_B) & 0xFF
    total = op1 + neg_B + 1
    result = total & 0xFF  # Truncate to 8 bits.
    # Discard the result, just compute the flags

    # Update flags passing op1 and neg_B (i.e., the effective second operand).
    self.update_flags(total, result, op1, neg_B)

@opcode(0x09)
def opcode_INR_A(self):
    print("Executing opcode: INR A")
    op1 = self.register_A
    total = op1 + 1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags with the original operands for addition.
    self.update_flags(total, result, op1, 1)

@opcode(0x0A)
def opcode_INR_B(self):
    print("Executing opcode: INR B")
    op1 = self.register_B
    total = op1 + 1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_B = result

    # Update flags with the original operands for addition.
    self.update_flags(total, result, op1, 1)

@opcode(0x0B)
def opcode_INR_C(self):
    print("Executing opcode: INR C")
    op1 = self.register_C
    total = op1 + 1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_C = result

    # Update flags with the original operands for addition.
    self.update_flags(total, result, op1, 1)

@opcode(0x0C)
def opcode_DCR_A(self):
    print("Executing opcode: DCR A")
    op1 = self.register_A
    op2 = 1
    # Compute two's complement of op2.
    neg_2 = (~op2) & 0xFF
    total = op1 + neg_2 + 1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and neg_B (i.e., the effective second operand).
    self.update_flags(total, result, op1, neg_2)

@opcode(0x0D)
def opcode_DCR_B(self):
    print("Executing opcode: DCR B")
    op1 = self.register_B
    op2 = 1
    # Compute two's complement of op2.
    neg_2 = (~op2) & 0xFF
    total = op1 + neg_2 + 1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_B = result

    # Update flags passing op1 and neg_B (i.e., the effective second operand).
    self.update_flags(total, result, op1, neg_2)

@opcode(0x0E)
def opcode_DCR_C(self):
    print("Executing opcode: DCR C")
    op1 = self.register_C
    op2 = 1
    # Compute two's complement of op2.
    neg_2 = (~op2) & 0xFF
    total = op1 + neg_2 + 1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_C = result

    # Update flags passing op1 and neg_B (i.e., the effective second operand).
    self.update_flags(total, result, op1, neg_2)

###########
### Logic
###########

@opcode(0x0F)
def opcode_ANA_B(self):
    print("Executing opcode: ANA B")
    op1 = self.register_A
    op2 = self.register_B
    total = op1 & op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x10)
def opcode_ANA_C(self):
    print("Executing opcode: ANA C")
    op1 = self.register_A
    op2 = self.register_C
    total = op1 & op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x11)
def opcode_TST(self):
    print("Executing opcode: TST")
    op1 = self.register_A
    op2 = self.register_B
    total = op1 & op2
    result = total & 0xFF  # Truncate to 8 bits.
    # Discard the result, just compute the flags

    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x12)
def opcode_ORA_B(self):
    print("Executing opcode: ORA B")
    op1 = self.register_A
    op2 = self.register_B
    total = op1 | op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x13)
def opcode_ORA_C(self):
    print("Executing opcode: ORA C")
    op1 = self.register_A
    op2 = self.register_C
    total = op1 | op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x14)
def opcode_XRA_B(self):
    print("Executing opcode: XRA B")
    op1 = self.register_A
    op2 = self.register_B
    total = op1 ^ op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x15)
def opcode_XRA_C(self):
    print("Executing opcode: XRA C")
    op1 = self.register_A
    op2 = self.register_C
    total = op1 ^ op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x16)
def opcode_CMA(self):
    print("Executing opcode: CMA")
    op1 = self.register_A
    op2 = 0x00
    total = ~op1
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result

    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x17)
def opcode_ANI(self):
    print("Executing opcode: ANI Byte")
    # The argument of this opcode is the immediate value with which we want to perform a logical AND
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address
    op1 = self.register_A
    op2 = self.memory[self.ip]   # The immediate value Byte
    total = op1 & op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result
 
    # We need to advance the instruction pointer
    self.ip += 1
 
    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x18)
def opcode_ORI(self):
    print("Executing opcode: ORI Byte")
    # The argument of this opcode is the immediate value with which we want to perform a logical OR
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address
    op1 = self.register_A
    op2 = self.memory[self.ip]   # The immediate value Byte
    total = op1 | op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result
 
    # We need to advance the instruction pointer
    self.ip += 1
 
    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)

@opcode(0x19)
def opcode_XRI(self):
    print("Executing opcode: XRI Byte")
    # The argument of this opcode is the immediate value with which we want to perform a logical XOR
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address
    op1 = self.register_A
    op2 = self.memory[self.ip]   # The immediate value Byte
    total = op1 ^ op2
    result = total & 0xFF  # Truncate to 8 bits.
    self.register_A = result
 
    # We need to advance the instruction pointer
    self.ip += 1
 
    # Update flags passing op1 and op2.
    self.update_flags(total, result, op1, op2)


##########################
### Register to register
##########################

@opcode(0x1A)
def opcode_MOV_A_B(self):
    print("Executing opcode: MOV A B")
    self.register_A = (self.register_B) & 0xFF
    # No flags are updated

@opcode(0x1B)
def opcode_MOV_A_C(self):
    print("Executing opcode: MOV A C")
    self.register_A = (self.register_C) & 0xFF
    # No flags are updated

@opcode(0x1C)
def opcode_MOV_B_A(self):
    print("Executing opcode: MOV B A")
    self.register_B = (self.register_A) & 0xFF
    # No flags are updated

@opcode(0x1D)
def opcode_MOV_B_C(self):
    print("Executing opcode: MOV B C")
    self.register_B = (self.register_C) & 0xFF
    # No flags are updated

@opcode(0x1E)
def opcode_MOV_C_A(self):
    print("Executing opcode: MOV C A")
    self.register_C = (self.register_A) & 0xFF
    # No flags are updated

@opcode(0x1F)
def opcode_MOV_C_B(self):
    print("Executing opcode: MOV C B")
    self.register_C = (self.register_B) & 0xFF
    # No flags are updated

@opcode(0x20)
def opcode_MVI_A_Byte(self):
    print("Executing opcode: MVI A Byte")
    # The argument of this opcode is the immediate value which we want to copy into A
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address    
    self.register_A = self.memory[self.ip] & 0xFF

    # We need to advance the instruction pointer
    self.ip += 1
    # No flags are updated

@opcode(0x21)
def opcode_MVI_B_Byte(self):
    print("Executing opcode: MVI B Byte")
    # The argument of this opcode is the immediate value which we want to copy into B
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address    
    self.register_B = self.memory[self.ip] & 0xFF

    # We need to advance the instruction pointer
    self.ip += 1
    # No flags are updated

@opcode(0x22)
def opcode_MVI_C_Byte(self):
    print("Executing opcode: MVI C Byte")
    # The argument of this opcode is the immediate value which we want to copy into C
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address    
    self.register_C = self.memory[self.ip] & 0xFF

    # We need to advance the instruction pointer
    self.ip += 1
    # No flags are updated


###################
### Memory access
###################

@opcode(0x23)
def opcode_LDA_Address(self):
    print("Executing opcode: LDA Address")
    # The argument of this opcode is the memory address from which we want to read.
    # The instruction pointer should point to the argument
    # This operation is not memory safe: no checks are done on the validity of the address
    address = self.memory[self.ip]
    self.register_A = self.memory[address]
    # We need to advance the instruction pointer
    self.ip += 1
    # No flags are updated

@opcode(0x24)
def opcode_LDB_Address(self):
    print("Executing opcode: LDB Address")
    # The argument of this opcode is the memory address from which we want to read.
    # The instruction pointer should point to the argument
    # This operation is not memory safe: no checks are done on the validity of the address
    address = self.memory[self.ip]
    self.register_B = self.memory[address]
    # We need to advance the instruction pointer
    self.ip += 1
    # No flags are updated

@opcode(0x25)
def opcode_LDC_Address(self):
    print("Executing opcode: LDC Address")
    # The argument of this opcode is the memory address from which we want to read.
    # The instruction pointer should point to the argument
    # This operation is not memory safe: no checks are done on the validity of the address
    address = self.memory[self.ip]
    self.register_C = self.memory[address]
    # We need to advance the instruction pointer
    self.ip += 1
    # No flags are updated

@opcode(0x26)
def opcode_STA_Address(self):
    print("Executing opcode: STA Address")
    # The argument of this opcode is the memory address to which we want to write.
    # The instruction pointer should point to the argument
    # This operation is not memory safe: no checks are done on the validity of the address
    address = self.memory[self.ip]
    self.memory[address] = self.register_A
    # We need to advance the instruction pointer
    self.ip += 1
    # No flags are updated

@opcode(0x29)
def opcode_LDA_C(self):
    print("Executing opcode: LDA C")
    # This operation is not memory safe: no checks are done on the validity of the address in register C
    address = self.register_C
    self.register_A = self.memory[address]
    # No flags are updated

@opcode(0x2A)
def opcode_STA_C(self):
    print("Executing opcode: STA C")
    # This operation is not memory safe: no checks are done on the validity of the address in register C
    address = self.register_C
    self.memory[address] = self.register_A
    # No flags are updated


##############
### Branches
##############

@opcode(0x2B)
def opcode_JMP(self):
    print("Executing opcode: JMP Address")
    # The argument of this opcode is the memory address to which we want to jump.
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address
    address = self.memory[self.ip]
    self.ip = address
    # No flags are updated

@opcode(0x2C)
def opcode_JS(self):
    print("Executing opcode: JS Address")
    # The argument of this opcode is the memory address to which we want to jump.
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address
    if self.flag_S == 1:
        address = self.memory[self.ip]
        self.ip = address
    else:
        self.ip += 1
    # No flags are updated

@opcode(0x2D)
def opcode_JZ(self):
    print("Executing opcode: JZ Address")
    # The argument of this opcode is the memory address to which we want to jump.
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address
    if self.flag_Z == 1:
        address = self.memory[self.ip]
        self.ip = address
    else:
        self.ip += 1
    # No flags are updated

@opcode(0x2E)
def opcode_JC(self):
    print("Executing opcode: JC Address")
    # The argument of this opcode is the memory address to which we want to jump.
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address
    if self.flag_C == 1:
        address = self.memory[self.ip]
        self.ip = address
    else:
        self.ip += 1
    # No flags are updated

@opcode(0x2F)
def opcode_JV(self):
    print("Executing opcode: JV Address")
    # The argument of this opcode is the memory address to which we want to jump.
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address
    if self.flag_V == 1:
        address = self.memory[self.ip]
        self.ip = address
    else:
        self.ip += 1
    # No flags are updated

##########################
### Call methods
##########################

@opcode(0x30)
def opcode_CALL(self):
    print("Executing opcode: CALL Address")
    # The argument of this opcode is the memory address where the routine is found
    # The instruction pointer should point to the argument (correct value set by the execution cycle)
    # This operation is not memory safe: no checks are done on the validity of the address
    address = self.memory[self.ip]      # The address where the routine is
    self.ip += 1                        # The address whre we want to return to
    self.memory[0xFF] = self.ip         # Store the return address at 0xFF
    self.ip = address                   # Jump to the routine
    # No flags are updated

@opcode(0x31)
def opcode_RET(self):
    print("Executing opcode: RET")
    # Return from a routine call
    # The return address is expected to be at position 0xFF
    # This operation is not memory safe: no checks are done on the validity of the address
    address = self.memory[0xFF]      # The return address
    self.ip = address                # Return!
    # No flags are updated

##########################
### Miscellaneous
##########################

@opcode(0x3E)
def opcode_NOP(self):
    print("Executing opcode: NOP")
    # No operation—just consume the cycle and return.
    pass


@opcode(0x3F)
def opcode_HLT(self):
    print("Executing opcode: HLT")
    # Just stop the machine. Flags are not touched
    self.halted = True

### Using the Assembler and the Emulator

**An example program**

In [ ]:
asm_program = """
    LDA x			; Load the value at address x into register A
    LDB y			; Load the value at address y into register B
    SUB B			; Perform the subtraction A = A - B
    CMA				; Complement A
    JS positiv		; Jump if sign flag is set (A>=B)
    STA d			; Store the value in register A to address d
    HLT				; Stop the program
    positiv:
    CMA				; Complement A
    STA d			; Store the value in register A to address d
    HLT				; Stop the program
    x:				; First number
    0x0A
    y:				; Second number
    0x03
    d:				; Result
    0x00
"""

print(asm_program)




**Assemble the program and obtain the machine code**

In [ ]:
assembler = SimpleAssembler()
mc_program, mc_listing = assembler.assemble_with_listing(asm_program)
print("Machine Code listing with memory adresses, memory content and comments:")
print(mc_listing)
print("")
print("The machine code:" )
print(mc_program)

**Run the program on the CPU Emulator**

In [ ]:
emulator = SimpleCPUEmulator()

emulator.read_into_memory(mc_program)

print(f"Initial state:")
emulator.display_current_state()

emulator.run()

print(f"Final state:")
emulator.display_current_state()
emulator.memory_dump()